In [150]:
from pdf2image import convert_from_path
import os
import json

In [151]:
functions = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "(3-1)+6*2+1"
		     },
		"correct": "(3-1)",
		"incorrect": ["6*2", "2+1"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso? a.Suma b.Multiplicación",
			"math": "2+6*2+1"
		     },
		"correct": "6*2",
		"incorrect": ["2+6", "2+1"]
	}
]

functions2 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero? a.Suma b.Paréntesis c.División",
			"math": "8/2(2+2)"
		     },
		"correct": "(2+2)",
		"incorrect": ["8/2"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso? a.División b.Multiplicación",
			"math": "8/2(4)"
		     },
		"correct": "8/2",
		"incorrect": ["2(4)"]
	}
]

functions3 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "3+5x4(4/4+1)"
		     },
		"correct": "4/4",
		"incorrect": ["3+5", "5*4", "4+1"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "3+5*4(1+1)"
		     },
		"correct": "1+1",
		"incorrect": ["3+5", "5*4"]
	},
 	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "3+5*4*2"
		     },
		"correct": "5*4*2",
		"incorrect": ["3+5"]
	}
]

functions4 = [
	{
		"q": {
			"text": "¿Qué parte de la siguiente ecuación debes resolver primero?",
			"math": "8+8(8*8)+5"
		     },
		"correct": "(8*8)",
		"incorrect": ["8+8", "8/8"]
	},
	{
		"q": {
			"text": "¿Cuál es el siguiente paso?",
			"math": "8+8*64+5"
		     },
		"correct": "8*64",
		"incorrect": ["8+8", "64+5"]
	}
]

In [152]:
def parse_latex(functions):
    levels = []
    for level, function in enumerate(functions):
        level_dict = {
            "question": {
                "type": "Image",
                "value": f"{level}/question.png"
            },
            "answers": [{
                "type": "Image",
                "value": f"{level}/correct.png",
                "answer": True
            }]
        }
        # Question
        qtex = f"""
                \\documentclass{{slides}}
                \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
                \\begin{{document}}
                \\begin{{Large}}
                \\begin{{center}}
                    {function["q"]["text"]}
                    \\[ {function["q"]["math"]} \\]
                \\end{{center}}
                \\end{{Large}}
                \\end{{document}}
                """
        with open("qtemp.tex", "w") as f:
            f.write(qtex)
            f.close()
        os.system("pdflatex qtemp.tex")
        qimage = convert_from_path("qtemp.pdf")
        os.mkdir(f"./game/assets/questions/{level}")
        qimage[0].save(f"./game/assets/questions/{level}/question.png", "PNG")

        # correct answer
        tex = f"""
              \\documentclass{{slides}}
              \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
              \\begin{{document}}
              \\begin{{huge}}
                \\[ {function['correct']} \\]
              \\end{{huge}}
              \\end{{document}}
              """
        with open("ctemp.tex", "w") as f:
            f.write(tex)
            f.close()
        os.system("pdflatex ctemp.tex")
        cimage = convert_from_path("ctemp.pdf")
        cimage[0].save(f"./game/assets/questions/{level}/correct.png", "PNG")

        # incorrect answers
        for i, answer in enumerate(function['incorrect']):
            incorrect_answer = {
                "type": "Image",
                "value": f"{level}/incorrect{i}.png",
                "answer": False
            }
            level_dict["answers"].append(incorrect_answer)
            tex = f"""
                  \\documentclass{{slides}}
                  \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
                  \\begin{{document}}
                  \\begin{{huge}}
                    \\[ {answer} \\]
                  \\end{{huge}}
                  \\end{{document}}
                  """
            with open("temp.tex", "w") as f:
                f.write(tex)
                f.close()
            os.system("pdflatex temp.tex")
            image = convert_from_path("temp.pdf")
            image[0].save(f"./game/assets/questions/{level}/incorrect{i}.png", "PNG")
        
        levels.append(level_dict)
    open("game/assets/questions/questions.json", "w").write(json.dumps(levels))


In [153]:
parse_latex(functions)